In [369]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import CubicSpline
from mplsoccer.pitch import Pitch
import json
import pandas as pd
import re
from math import isnan
from datetime import datetime, timedelta
%run smoothing.ipynb


In [370]:
import csv

def read_csv_partially(file_path, start_line, end_line):
    data = []  # List to store extracted CSV rows

    with open(file_path, 'r', newline='') as file:
        reader = csv.reader(file)
        for line_num, row in enumerate(reader):
            if line_num >= start_line and line_num <= end_line:
                # Append the row to data
                data.append(row)
            elif line_num > end_line:
                break  # Stop reading once end_line is reached

    return data

In [371]:
file_path_skillcorner = '../data/matches/vs FC Lugano/Preprocessed Skillcorner data/1296476_tracking.csv'
match_info_path = '../data/matches/vs FC Lugano/Skillcorner data/1296476_match_info.json'
partialframes = read_csv_partially(file_path_skillcorner, 0, 100000)

match_info = json.load(open (match_info_path))
# targetIDSkillcorner = 7679
targetIDSkillcorner = 10218
targetPositionsSkillcorner = []

playerIDs = set()
teamID = 293

for player in list(filter(lambda player: player['team_id'] == teamID, match_info['players'])):
    playerIDs.add(str(player['id']))

print(len(playerIDs))

targetTrajs = {}
playerSmoothDataFrames = pd.DataFrame()
for ID in playerIDs:
    targetTrajs[ID] = []
    playerSmoothDataFrames = pd.DataFrame()

# for index, frame in enumerate(partialframes):
#     if(index % 1 != 0): # We consider only every n-th frame
#         continue

#     if(frame['player_data'] == []):
#         continue

#     player = list(filter(lambda player: player['player_id'] == targetIDSkillcorner, frame['player_data']))[0]

#     # defining datapoints d_i = [x_Coordinate, y_Coordinate, speed, time]
#     targetPositionsSkillcorner.append([player['x'], player['y'], 0, datetime.strptime(frame['timestamp'], "%H:%M:%S.%f")])

print(playerIDs)
for index, playerFrame in enumerate(partialframes):
    # print(playerFrame)

    if(playerFrame[4] not in playerIDs or playerFrame[8] == 'True'):
        continue

    # defining datapoints d_i = [x_Coordinate, y_Coordinate, speed, time]
    if targetTrajs[playerFrame[4]] == [] or int(targetTrajs[playerFrame[4]][-1][-1][-1]) != (int(playerFrame[2]) - 1):
        targetTrajs[playerFrame[4]].append([])
        
    targetTrajs[playerFrame[4]][-1].append([float(playerFrame[5]), float(playerFrame[6]), 0, datetime.fromtimestamp(int(playerFrame[3])/1000.0), int(playerFrame[2])])

print(targetTrajs['16280'])


20
{'584967', '102835', '689056', '36724', '14595', '19420', '3233', '10218', '659794', '127332', '59893', '666916', '7679', '162993', '7446', '24934', '16280', '530096', '43052', '258924'}
[[[12.02, 13.05, 0, datetime.datetime(1970, 1, 1, 1, 0), 100], [12.24, 13.05, 0, datetime.datetime(1970, 1, 1, 1, 0, 0, 100000), 101], [12.45, 13.05, 0, datetime.datetime(1970, 1, 1, 1, 0, 0, 200000), 102], [12.63, 13.05, 0, datetime.datetime(1970, 1, 1, 1, 0, 0, 300000), 103], [12.79, 13.06, 0, datetime.datetime(1970, 1, 1, 1, 0, 0, 400000), 104], [12.94, 13.07, 0, datetime.datetime(1970, 1, 1, 1, 0, 0, 500000), 105]], [[14.37, 13.19, 0, datetime.datetime(1970, 1, 1, 1, 0, 3, 700000), 137], [14.29, 13.19, 0, datetime.datetime(1970, 1, 1, 1, 0, 3, 800000), 138], [14.2, 13.19, 0, datetime.datetime(1970, 1, 1, 1, 0, 3, 900000), 139], [14.09, 13.19, 0, datetime.datetime(1970, 1, 1, 1, 0, 4), 140], [13.95, 13.19, 0, datetime.datetime(1970, 1, 1, 1, 0, 4, 100000), 141], [13.8, 13.2, 0, datetime.datetime(

In [372]:
# pitch = Pitch(axis=True, label=True, tick=True, pitch_type='skillcorner', pitch_length=105, pitch_width=68)
# fig, ax = pitch.draw(figsize=(10, 20))

# x, y, s, t = zip(*targetPositionsSkillcorner)
# ax.scatter(x, y, color='black', marker='o', s=20)

# ax.set_xlim(-5, 20) 
# ax.set_ylim(0, 20)

# add for loop that starts here and smootens every trajectory in the traj list
playerSmoothDataFrames = pd.DataFrame()

for player, trajs in targetTrajs.items():
    if player == []:
        continue

    for traj in trajs:
        if(len(traj) < 2):
            continue

        new_traj = performSmoothing(datapoints=traj, iterations=50)
        new_traj = pd.DataFrame(new_traj)
        new_traj[5] = player
        traj = pd.DataFrame(traj)
        for idx, row in traj.iterrows():
            if(isnan(row[4])):
                print('nan')

        new_traj[4] = traj[4]

        playerSmoothDataFrames = pd.concat([playerSmoothDataFrames, new_traj], axis=0, ignore_index=True)


originalDF = pd.DataFrame(partialframes[1:])

# for frame in playerSmoothDataFrames['16280'][4]:
#     print(frame)

playerSmoothDataFrames[4] = playerSmoothDataFrames[4].astype(int)
# print(playerSmoothDataFrames['16280'])
playerSmoothDataFrames.columns = ['x', 'y', 'speed', 'time', 'frame_id', 'object_id']
originalDF.columns = ['match_id', 'half', 'frame_id', 'timestamp', 'object_id', 'x', 'y', 'z', 'extrapolated']

originalDF['frame_id'] = originalDF['frame_id'].astype(int)
# print(originalDF)
# print('----------------------------------------------')
print(type(playerSmoothDataFrames['frame_id'][0]))
print(type(originalDF['frame_id'][0]))
newDF = originalDF.merge(playerSmoothDataFrames, how='left', on=['object_id', 'frame_id'])

# traj = performSmoothing(datapoints=targetPositionsSkillcorner, iterations=10)
# x, y, s, t = zip(*(traj))

# for idx, t in enumerate(traj):
#     ax.scatter(t[0], t[1], color="purple", marker="o", s=10)
#     # ax.text(t[0], t[1], f'(ss:{"{:.2f}".format(t[2])}) og: {"{:.2f}".format(targetPositionsSkillcorner[idx][2])}')

# plt.show()

# for index, frame in enumerate(partialframes):
#     # add smoothened frames, invariant: Frames are continuous from start to finish, so you have to only remember start and finish
#     frame['player_data'][0] = {
#         "x":1000,
#          "y":1000,
#         "player_id":30961,
#         "is_detected": True
#     }

# trajcetory smoothing for loop ends here 

# print(partialframes[0])

<class 'numpy.int64'>
<class 'numpy.int64'>


In [373]:
# print("Before transformation:")
# print(newDF.iloc[180])
newDF
# print(newDF.loc[300, 'x_y'])
# print(newDF.loc[300, 'y_y'])
# print(newDF.loc[300, 'x_x'])
# print(newDF.loc[300, 'y_x'])
# print("After transformation:")
# print(newDF.iloc[180])

,match_id,half,frame_id,timestamp,object_id,x_x,y_x,z,extrapolated,x_y,y_y,speed,time
0,1296476,1,100,0,-1,0.46,0.3,0.17,True,NaN,NaN,NaN,NaT
1,1296476,1,100,0,34469,-39.43,0.59,0.0,True,NaN,NaN,NaN,NaT
2,1296476,1,100,0,22148,-20.81,-0.41,0.0,True,NaN,NaN,NaN,NaT
3,1296476,1,100,0,59888,-18.07,-11.77,0.0,True,NaN,NaN,NaN,NaT
4,1296476,1,100,0,33868,-18.61,12.24,0.0,False,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1296476,1,4821,472100,162993,16.4,-7.59,0.0,True,NaN,NaN,NaN,NaT
99996,1296476,1,4821,472100,16280,16.6,8.93,0.0,True,NaN,NaN,NaN,NaT
99997,1296476,1,4821,472100,59893,16.15,-18.02,0.0,True,NaN,NaN,NaN,NaT
99998,1296476,1,4821,472100,127332,8.58,-15.95,0.0,True,NaN,NaN,NaN,NaT


In [374]:
newDF['x_x'] = newDF['x_y'].fillna(newDF['x_x'])
newDF['y_x'] = newDF['y_y'].fillna(newDF['y_x'])

newDF.drop(columns=['x_y', 'y_y', 'speed', 'time'], inplace=True)

newDF


,match_id,half,frame_id,timestamp,object_id,x_x,y_x,z,extrapolated
0,1296476,1,100,0,-1,0.46,0.3,0.17,True
1,1296476,1,100,0,34469,-39.43,0.59,0.0,True
2,1296476,1,100,0,22148,-20.81,-0.41,0.0,True
3,1296476,1,100,0,59888,-18.07,-11.77,0.0,True
4,1296476,1,100,0,33868,-18.61,12.24,0.0,False
...,...,...,...,...,...,...,...,...,...
99995,1296476,1,4821,472100,162993,16.4,-7.59,0.0,True
99996,1296476,1,4821,472100,16280,16.6,8.93,0.0,True
99997,1296476,1,4821,472100,59893,16.15,-18.02,0.0,True
99998,1296476,1,4821,472100,127332,8.58,-15.95,0.0,True


In [375]:
import pandas as pd

# Example DataFrames
df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
df2 = pd.DataFrame()

# Extend df1 with df2
extended_df = pd.concat([df1, df2], ignore_index=True)
extended_df = pd.concat([extended_df, df1])

# Display the extended DataFrame
print(extended_df)

   A  B
0  1  4
1  2  5
2  3  6
0  1  4
1  2  5
2  3  6
